<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
    
Cette étude a été réalisée dans le cadre du 6ème projet de ma formation Datascientist dispensée en MOOC par 

<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>.




<p></p><p></p><p></p>





**Le problème posé :**


*Stack Overflow est un site célèbre de question-réponses liées au développement informatique. Pour poser une question sur ce site, il faut entrer plusieurs tags de manière à retrouver facilement la question par la suite. Pour les utilisateurs expérimentés cela ne pose pas de problème, mais pour les nouveaux utilisateurs, il serait judicieux de suggérer quelques tags relatifs à la question posée.*

*Amateur de Stack Overflow, qui vous a souvent sauvé la mise, vous décidez d'aider la communauté en retour. Pour cela, vous développez un système de suggestion de tags pour le site. Celui-ci prendra la forme d’un algorithme de machine learning qui assigne automatiquement plusieurs tags pertinents à une question.*


In [ ]:
from P6_PostClassifier import *
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
import p5_util

file_name="./data/train_StackOverFlow_BodyTitleTags.csv"
ser_sof_tags_train=pd.read_csv(file_name)['Tags']
print(ser_sof_tags_train.shape)

# List of referenced TAG is built...
list_list_tags_train  \
= ser_sof_tags_train.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>').tolist()
print(len(list_list_tags_train))

list_ref_tags = list()
for list_tags in list_list_tags_train :
    list_ref_tags += [tag for tag in list_tags]
    
list_unique_tags_train = list(set(list_ref_tags))
print(len(list_unique_tags_train))

filename='./data/list_unique_tags_train'
p5_util.object_dump(list_unique_tags_train,filename)

In [ ]:
import pandas as pd

file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
df_sof_test=pd.read_csv(file_name)

## <font color='blus'> 1.1 Loading LDA model</font>

In [ ]:
from P6_PostClassifier import *

oP6_PostClassifier = P6_PostClassifier(path_to_model='./data')
oP6_PostClassifier.show()

### 1.1.1 LDA model configuration

In [ ]:
import p5_util

model_name_file = 'lda_bow_400topics.dump'
model_name = 'LDA'
vectorizer_name = 'BOW'
vectorizer_file_name="vectorizer_bow.dump"
oP6_PostClassifier.load_model_from_file_name(model_name, model_name_file, vectorizer_name,vectorizer_file_name )
oP6_PostClassifier.list_tag_ref = list_unique_tags_train


oP6_PostClassifier.model_name='LDA'
oP6_PostClassifier.df_validation = df_sof_test
oP6_PostClassifier.show()

p5_util.object_dump(oP6_PostClassifier,'./data/oP6_PostClassifier.dump')

In [ ]:
oP6_PostClassifier.df_validation.shape

In [ ]:
df_sample = df_sof_test.sample()
body = df_sample.Body.iloc[0]
title=df_sample.Title.iloc[0]
tags = df_sample.Tags.iloc[0]

oP6_PostClassifier.suggest(body, title, tags)

In [ ]:
print()
print(title)
print()
print(body)

In [ ]:
list_tag_suggested = ['java', 'php', 'system', 'specific', 'perl', 'work', 'everything']

In [ ]:
list_tag_fw = list()
for word in list_tag_suggested :

    list_tuple_score = process.extract(word, oP6_PostClassifier._list_tag_ref)

    list_tag_fw += [tuple_score[0] for tuple_score \
    in list_tuple_score if tuple_score[1] >= 95]
list_tag_fw

## <font color='blus'> 1.2. Loading Logistic Regression model</font>

In [ ]:
from P6_PostClassifier import *

model_name_file = 'classifier_best_logisticRegression.dump'
model_name = 'LogReg'
vectorizer_name = 'TFIDF'
vectorizer_file_name="vectorizer_tfidf_ngram1_0.0002_.dump"

oP6_PostClassifier = P6_PostClassifier(path_to_model='./data')

oP6_PostClassifier.load_model_from_file_name(model_name, model_name_file, vectorizer_name,vectorizer_file_name )
oP6_PostClassifier.list_tag_ref = list_unique_tags_train
oP6_PostClassifier.model_name=model_name
oP6_PostClassifier.show()

In [ ]:
df_sample = df_sof_test.sample()
body = df_sample.Body.iloc[0]
title=df_sample.Title.iloc[0]
tags = df_sample.Tags.iloc[0]

list_tag_suggested, list_tag_suggested_fw, list_assigned_tags = oP6_PostClassifier.suggest(body,title,tags)
list_tag_suggested=list()
X = oP6_PostClassifier._dict_vectorizer['TFIDF'].transform(oP6_PostClassifier._ser_post)

y_pred = oP6_PostClassifier._dict_model['LogReg'].predict(X)
print(X.shape, y_pred.A.shape)
dict_row_col_pred = p6_util.p6_get_dict_row_col_from_csrmatrix(y_pred)

for row, list_col_pred in dict_row_col_pred.items() :
    list_col_pred = dict_row_col_pred[row]
    print(list_col_pred)

list_tag_suggested = list()
for col in list_col_pred:
    tag_suggested = oP6_PostClassifier._list_tag_ref[col]
    list_tag_suggested.append(tag_suggested)
print(list_tag_suggested,list_assigned_tags)
print()
print(title)
print()
print(body)

In [ ]:
np.where(y_pred.A >0)
#dict_row_col_pred

In [ ]:
tuple_index = np.where(y_pred.A >0)
oP6_PostClassifier._ser_post
print(oP6_PostClassifier._list_tag_ref[575],oP6_PostClassifier._list_tag_ref[4910])


In [ ]:
tags

## <font color='blus'> 1.3. Loading KRR Gaussian model</font>

In [ ]:
oP6_PostClassifier = P6_PostClassifier(path_to_model='./data')
oP6_PostClassifier.show()

In [ ]:
oP6_PostClassifier.list_tag_ref=list_unique_tags_train
oP6_PostClassifier.show()

In [ ]:
model_name_file = 'classifier_KRRGaussian.dump'
model_name = 'KRR'

vectorizer_name = 'TFIDF'
vectorizer_file_name="vectorizer_tfidf_ngram1_0.0002_.dump"

oP6_PostClassifier.load_model_from_file_name(model_name, model_name_file, vectorizer_name,vectorizer_file_name )
oP6_PostClassifier.model_name = model_name
oP6_PostClassifier.show()

In [ ]:
df_sample = df_sof_test.sample()
body = df_sample.Body.iloc[0]
title=df_sample.Title.iloc[0]
tags = df_sample.Tags.iloc[0]

oP6_PostClassifier.suggest(body,title,tags)

# <font color='blus'> 2.0 Loading oP6_PostClassifier</font>

In [1]:
from P6_PostClassifier import *
import P6_PostClassifier

oP6_PostClassifier = P6_PostClassifier.load_dumped()
oP6_PostClassifier.show()

./data/oP6_PostClassifier.dump

  
Verbose  ................: True
Path model name  ........: ./data
Model name ..............: LDA
Model ...................: LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=400, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

Vectorizer name ..............: BOW
Vectorizer ...................: CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.1, max_features=None, min_df=0.001,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        toke

/home/bangui/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.20.0 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/bangui/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.0 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [2]:
list_tag_suggested, list_tag_suggested_fw, list_assigned_tags, body, title = oP6_PostClassifier.process_post(None)
json_result = oP6_PostClassifier.json_builder(list_tag_suggested, list_tag_suggested_fw, list_assigned_tags, body, title)
print(json_result)
print()



{"_result":[{tag_s : ['class']},{tag_a : ['c++', 'oop', 'visual-c++', 'templates']},{title : is it possible to have templated classes within a template class?},{body : <pre><code>template &lt;class M, class A&gt; class C { std::list&lt;M&gt; m_List; ... }
</code></pre>

<p>Is the above code possible?  I would like to be able to do something similar.</p>

<p>Why I ask is that i get the following error:</p>

<pre><code>Error 1 error C2079: 'std::_List_nod&lt;_Ty,_Alloc&gt;::_Node::_Myval' uses undefined class 'M'   C:\Program Files\Microsoft Visual Studio 9.0\VC\include\list    41
</code></pre>
},]}



In [ ]:
print()

print(len(oP6_PostClassifier.df_validation.Body[post_id]))

In [ ]:
df_sof_test.iloc[10]

In [ ]:
post_id=0
body = df_sof_test.Body.iloc[post_id]
title = df_sof_test.Title.iloc[post_id]
tag_a = df_sof_test.Tags.iloc[post_id]
print()
print(tag_a)
print()
print(title)
print()
print(body)

In [ ]:
oP6_PostClassifier.df_validation.shape